<a href="https://colab.research.google.com/github/jakubpiwowarski/Poker_Hand_Detector/blob/main/Poker_Hand_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
from google.colab.patches import cv2_imshow
!pip install ultralytics==8.0.28

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.28 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.2/78.2 GB disk)


In [4]:
from ultralytics import YOLO

In [5]:
pip install cvzone


  Preparing metadata (setup.py) ... done
  Created wheel for cvzone: filename=cvzone-1.5.6-py3-none-any.whl size=18747 sha256=74a1e617eb7b8f069a82617dd721646f5e10e8901962d1878c633596aa6df79d
  Stored in directory: /root/.cache/pip/wheels/d8/85/f1/1756f9e009d280be742fb20dd5087c60c2f7f0279964934375
Successfully built cvzone


In [6]:
import cvzone
import math
import cv2

In [7]:
!pip install opencv-python-headless
!pip install google-colab

In [8]:
pip install filterpy==1.4.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=9198921ab66609225dbb77f13eb1c50998a0b3e0fa69600ba1e3ae5077d96c87
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy


In [9]:
pip install scikit-image

In [10]:
pip install lap==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lap: filename=lap-0.4.0-cp310-cp310-linux_x86_64.whl size=1655216 sha256=8fab478a91d6d168e5d6335d2a487106901cd67db7219aeceecac222a1471ebe
  Stored in directory: /root/.cache/pip/wheels/00/42/2e/9dfe19270eea279d79e84767ff0d7b8082c3bf776cad00e83d
Successfully built lap


In [11]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/101-Object_Detection/Pliki/Project_4-Poker_Hand_Detector')

In [12]:
sys.path.append('/content/drive/MyDrive/101 - Object Detection /Pliki z kursu/Project 1 - Car Counter/sort.py')

In [13]:
sys.path.append('/content/drive/MyDrive/101-Object_Detection/Pliki/Project_4-Poker_Hand_Detector/pokerHandDetectorFunctions.py')

In [14]:
!cp -r "/content/drive/MyDrive/101-Object_Detection/Pliki/Project 1 - Car Counter/sort.py" '/content/'

In [15]:
!cp -r "/content/drive/MyDrive/101-Object_Detection/Pliki/Project_4-Poker_Hand_Detector/pokerHandDetectorFunctions.py" '/content/'

In [16]:
from sort import *

In [17]:
import pokerHandDetectorFunctions

In [18]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [19]:
model= YOLO('/content/drive/MyDrive/101-Object_Detection/Pliki/Project_4-Poker_Hand_Detector/playingCards.pt')

In [20]:
classNames = ['10C', '10D', '10H', '10S', '2C', '2D', '2H', '2S',
              '3C', '3D', '3H', '3S', '4C', '4D', '4H', '4S',
              '5C', '5D', '5H', '5S', '6C', '6D', '6H', '6S',
              '7C', '7D', '7H', '7S', '8C', '8D', '8H', '8S',
              '9C', '9D', '9H', '9S', 'AC', 'AD', 'AH', 'AS',
              'JC', 'JD', 'JH', 'JS', 'KC', 'KD', 'KH', 'KS',
              'QC', 'QD', 'QH', 'QS']


In [ ]:
while True:
    success, img = cap.read()
    results = model(img,stream=True)
    hand =[]
    for r in results:
        boxes = r.boxes
        for box in boxes:
            #bounding box
            x1,y1,x2,y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1),int(y1),int(x2),int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
            w, h = x2-x1, y2-y1
            bbox = int(x1), int(y1), int(x2), int(y2)
            cvzone.cornerRect(img,(x1,y1,w,h))
            #confidence
            conf = math.ceil((box.conf[0]*100))/100
            print(conf)

            #clas name
            cls = int(box.cls[0])
            cvzone.putTextRect(img, f'{classNames[cls]} {conf}', (max(0, x1), max(35, y1)),scale=2,thickness=2)

            if conf > 0.5:
                hand.append(classNames[cls])


    print(hand)
    hand = list(set(hand))
    if len(hand) == 5:
      results = pokerHandDetectorFunctions.findPokerHand(hand)
    cvzone.putTextRect(img, f'Your hand:{results}', (300,75),scale=3,thickness=5)
    cv2_imshow(img)
    cv2.waitKey(1)
